In [2]:
import numpy as np
import pandas as pd 

In [ ]:
# Modüllerin eklenmesi

import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import folium
from folium.plugins import HeatMap

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

degerlendirme_tablosu = pd.DataFrame({'Model': [],
                           'Detaylar':[],
                           'Root Mean Squared Error (RMSE)':[],
                           'R-squared (eğitim)':[],
                           'Adjusted R-squared (eğitim)':[],
                           'R-squared (test)':[],
                           'Adjusted R-squared (test)':[],
                           '5-Fold Cross Validation':[]})

df = pd.read_csv('../datasets/kc_house_data.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

# Adjusted $R^{2}$ Hesaplama Fonksiyonunu Tanımlama

Özellik sayısı arttıkça R-squared'da artar. Bundan dolayı, iki farklı modelin performansı karşılaştırmak için daha sağlam bir değerlendirici tercih edilir. Bu değerlendiriciye Adjusted $R^{2}$ denir ve yalnızca değişkenin eklenmesi MSE'yi azaltırsa artar.

Adjusted $R^{2}:$

$$\bar{R^{2}}=R^{2}-\frac{k-1}{n-k}(1-R^{2})$$

* n : gözlem sayısıdır.
* k : parametre sayısıdır.

In [ ]:
def adjustedR2(r2,n,k):
    return r2-(k-1)/(n-k)*(1-r2)

# Basit Doğrusal Regresyon Oluşturma:

Bir cevap ve sadece bir açıklayıcı değişken arasındaki doğrusal bir ilişkiyi modellediğimizde buna basit doğrusal regresyon denir. Konut fiyatlarını tahmin etmeye çalıştığımız için cevap değişkenimiz fiyattır(price). Basit bir model için bir tane özellik seçmemiz gerekir. Correlation matrisine baktığımız da fiyat ile ilişkili en iyi değişkenin sqft_living olduğunu görürüz. Bu sebeple özellik olarak sqft_living seçiyoruz.

In [ ]:
df.corr()["price"].sort_values(ascending=False)[:5]

In [ ]:
train_data,test_data = train_test_split(df,train_size = 0.8,random_state=3) # veri setimizi,eğitim seti ve test seti olarak 2'ye bölüyoruz.

lin_reg = linear_model.LinearRegression()

X_train = np.array(train_data['sqft_living'], dtype=pd.Series).reshape(-1,1) # özellik olarak sqft_living seçiyoruz ve X_train değişkenine atıyoruz.
y_train = np.array(train_data['price'], dtype=pd.Series) # hedef değer(yada cevap değer) olarak price seçiyoruz ve y_train değişkenine atıyoruz.
lin_reg.fit(X_train,y_train) # basit doğrusal regresyon modelimizi fit(uydurma) ediyoruz.

# test verisinde sqft_living özelliğini(açıklayıcı özellik) ve price özelliğini(cevap özellik) ayırıyoruz:
X_test = np.array(test_data['sqft_living'], dtype=pd.Series).reshape(-1,1) 
y_test = np.array(test_data['price'], dtype=pd.Series)

tahmin = lin_reg.predict(X_test) # test setindeki sqft_living özelliğini kullanarak,
                                 # oluşturduğumuz basit doğrusal regresyon modeline tahminler yaptırıyoruz.

rmsesm = float(format(np.sqrt(metrics.mean_squared_error(y_test,tahmin)),'.3f')) # mean squared error(ortalama kare hatası) hesaplıyoruz.
rtrsm = float(format(lin_reg.score(X_train, y_train),'.3f')) # modelin eğitim setindeki skorunu hesaplıyoruz.
rtesm = float(format(lin_reg.score(X_test, y_test),'.3f')) # modelin test setindeki skorunu hesaplıyoruz.
cv = float(format(cross_val_score(lin_reg,df[['sqft_living']],df['price'],cv=5).mean(),'.3f')) # cross validation(çapraz değerlendirme) yaparak 
                                                                                               # modelimizi değerlendiriyoruz.
    
print ("Test Setinde Ortalama Fiyat: {:.3f}".format(y_test.mean()))
print('Intercept(modelin y eksenini kestiği nokta): {}'.format(lin_reg.intercept_))
print('Coefficient(sqft_living özelliğinin katsayısı): {}'.format(lin_reg.coef_))

r = degerlendirme_tablosu.shape[0]
degerlendirme_tablosu.loc[r] = ['Basit Doğrusal Regresyon','-',rmsesm,rtrsm,'-',rtesm,'-',cv]
degerlendirme_tablosu

# Sonuca Bakalım

Modelimiz çok iyi uymuş(fit) gibi görünmüyor.

In [ ]:
plt.figure(figsize=(6.5,5))
plt.scatter(X_test,y_test,color='blue',label="Data", alpha=.1)
plt.plot(X_test,lin_reg.predict(X_test),color="red",label="Regresyon Tahmin Çizgisi")
plt.xlabel("Yaşam Alanı (sqft)", fontsize=15)
plt.ylabel("Fiyat ($)", fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.grid(True)
plt.legend()

plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

# Verileri Görselleştirme ve İnceleme

Bir model geliştirmeden önce grafik çizmek ve verileri incelemek çok faydalıdır,çünkü bazı olası aykırı değerleri tespit edebilir veya normalleştirmeye yapmaya karar verebiliriz. Bu bir zorunluluk değildir ancak verilerinizin her zaman iyi olduğunu bilmelisiniz. Dataframe'in histogramını çizerek başlayalım.

In [ ]:
df1=df[['price', 'bedrooms', 'bathrooms', 'sqft_living',
    'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
    'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
    'lat', 'long', 'sqft_living15', 'sqft_lot15']]
h = df1.hist(bins=25,figsize=(16,16),xlabelsize='10',ylabelsize='10',xrot=-15)
sns.despine(left=True, bottom=True)
[x.title.set_size(12) for x in h.ravel()];
[x.yaxis.tick_left() for x in h.ravel()];

Aşağıdaki grafiklerden, 33 yatak odası veya 7000000 civarında fiyat gibi diğerlerinden çok uzak bazı özelliklere veya fiyatlara sahip çok az sayıda ev olduğu görülebilir. Ancak, olası olumsuz etkilerinin belirlenmesi zaman alıcı ve gerçek veri setlerinde olacaktır. Bu veri kümesinde her zaman bazı lüks ev fiyatları gibi bazı aykırı değerler olacaktır. Bu yüzden aykırı değerleri kaldırmayı düşünmüyorum.

In [ ]:
sns.set(style="whitegrid", font_scale=1)
f, axes = plt.subplots(1, 2,figsize=(15,5))
sns.boxplot(x=df['bedrooms'],y=df['price'], ax=axes[0])
sns.boxplot(x=df['floors'],y=df['price'], ax=axes[1])
sns.despine(left=True, bottom=True)
axes[0].set(xlabel='Bedrooms - yatak odaları', ylabel='Price')
axes[0].yaxis.tick_left()
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='Floors - katlar', ylabel='Price')

f, axe = plt.subplots(1, 1,figsize=(12.18,5))
sns.despine(left=True, bottom=True)
sns.boxplot(x=df['bathrooms'],y=df['price'], ax=axe)
axe.yaxis.tick_left()
axe.set(xlabel='Banyo / Yatak Odası', ylabel='Price');

Yukarıdaki grafiklerden de görüldüğü gibi, yukarıda kullandığımız özellikler ile fiyat arasında mükemmel bir doğrusal ilişki yok.Peki diğer özellikler ile arasındaki ilişki? Bunu görstermek için 3D grafikler kullanalım. Nokta rengi olarak açık yeşil kullanılmıştır. Koyu yeşil kısımlar yüksek yoğunluk anlamına gelir,birçok açık yeşil nokta çakışır ve koyulaşır.

Aşağıdaki grafikler sqrt_living arttıkça,sqrt_lot ve bedrooms veya bathromms/bedrooms özelliklerinin de arttığını gösteriyor. Ancak floors,bedrooms ve bathrooms/bedrooms veya sqrt_living  benzer ilişkiye sahip değiller.

In [ ]:
fig=plt.figure(figsize=(19,12.5))
ax=fig.add_subplot(2,2,1, projection="3d")
ax.scatter(df['floors'],df['bedrooms'],df['bathrooms'],c="darkgreen",alpha=.5)
ax.set(xlabel='\nFloors',ylabel='\nBedrooms',zlabel='\nBathrooms / Bedrooms')
ax.set(ylim=[0,12])

ax=fig.add_subplot(2,2,2, projection="3d")
ax.scatter(df['floors'],df['bedrooms'],df['sqft_living'],c="darkgreen",alpha=.5)
ax.set(xlabel='\nFloors',ylabel='\nBedrooms',zlabel='\nsqft Living')
ax.set(ylim=[0,12])

ax=fig.add_subplot(2,2,3, projection="3d")
ax.scatter(df['sqft_living'],df['sqft_lot'],df['bathrooms'],c="darkgreen",alpha=.5)
ax.set(xlabel='\n sqft Living',ylabel='\nsqft Lot',zlabel='\nBathrooms / Bedrooms')
ax.set(ylim=[0,250000])

ax=fig.add_subplot(2,2,4, projection="3d")
ax.scatter(df['sqft_living'],df['sqft_lot'],df['bedrooms'],c="darkgreen",alpha=.5)
ax.set(xlabel='\n sqft Living',ylabel='\nsqft Lot',zlabel='Bedrooms')
ax.set(ylim=[0,250000]);

Daha fazla özelliği görselleştirelim. Aşağıdaki boxplot lara baktığımızda grade ve waterfront özelliğinin price özelliğini etkilediği açıkça görülebiliyor. Diğer tarafran view özelliği daha az etkilidir ancak yine de fiyat üzerinde etkisi vardır.

In [ ]:
f, axes = plt.subplots(1, 2,figsize=(15,5))
sns.boxplot(x=df['waterfront'],y=df['price'], ax=axes[0])
sns.boxplot(x=df['view'],y=df['price'], ax=axes[1])
sns.despine(left=True, bottom=True)
axes[0].set(xlabel='Waterfront', ylabel='Price')
axes[0].yaxis.tick_left()
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='View', ylabel='Price')

f, axe = plt.subplots(1, 1,figsize=(12.18,5))
sns.boxplot(x=df['grade'],y=df['price'], ax=axe)
sns.despine(left=True, bottom=True)
axe.yaxis.tick_left()
axe.set(xlabel='Grade', ylabel='Price');

view,grade ve year built arasındaki ilişkiyi belirlemek için 3D grafik çizdirelim. Aşağıdaki grafik, yeni evlerin daha iyi grade özelliği olduğunu ancak view özelliğindeki değişiklik hakkında çok fazla şey söyleyemeyeceğimizi gösteriyor.

In [ ]:
fig=plt.figure(figsize=(9.5,6.25))
ax=fig.add_subplot(1,1,1, projection="3d")
ax.scatter(train_data['view'],train_data['grade'],train_data['yr_built'],c="darkgreen",alpha=.5)
ax.set(xlabel='\nView',ylabel='\nGrade',zlabel='\nYear Built');

# Açıklayıcı Değişkenler Arasındaki Korelasyonun Gözden Geçirilmesi

Bir modelde çok fazla özelliğe sahip olmak her zaman iyi bir şey değildir çünkü aşırı uymaya(overfitting) ve yeni veri seti için değerler tahmin etmek istediğimizde kötü sonuçlar almamıza neden olabilir. Bu nedenle, bir özellik eğer modelinizi çok geliştirmiyorsa o özelliği eklememek daha iyidir.

Önemli olan başka bir şey de korelasyondur. İki özellik arasında yüksek korelasyon varsa, aşırı uymaya(overfitting) neden olmaması için o iki özelliğide tutmak çoğu zaman iyi bir fikir değildir. Örneğin, eğer aşırı uyma(overfitting) varsa,sqt_above veya sqt_living özelliğini kaldırabiliriz çünkü bu iki özellik yüksek korelasyonludur.Bu ilişki veri kümesindeki tanımlara baktığımızda tahmin edilebilir, ancak yine de korelasyon matrisi kontrol edilmelidir.Bununla birlikte, bu, yüksek derecede ilişkili özelliklerden birini kaldırmanız gerektiği anlamına gelmez. Örneğin:bathrooms ve sqrt_living. Bu iki özellik yüksek derecede ilişkilidirler(correlated) ancak aralarındaki ilişkinin sqt_living ve sqt_above arasındaki ilişki ile aynı olduğunu düşünmüyorum.

In [ ]:
features = ['price','bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront',
            'view','condition','grade','sqft_above','sqft_basement','yr_built','yr_renovated',
            'zipcode','lat','long','sqft_living15','sqft_lot15']

mask = np.zeros_like(df[features].corr(), dtype=np.bool) 
mask[np.triu_indices_from(mask)] = True 

f, ax = plt.subplots(figsize=(16, 12))
plt.title('Pearson Correlation Matrix',fontsize=25)

sns.heatmap(df[features].corr(),linewidths=0.25,vmax=0.7,square=True,cmap="BuGn", #"BuGn_r" to reverse 
            linecolor='w',annot=True,annot_kws={"size":8},mask=mask,cbar_kws={"shrink": .9});

# Veri Önişleme

Veriler üzerinde ön işleme yapmak modelin doğruluğunu arttırabilir ve modeli daha güvenli hale getirir. Bu işlem her zaman sonuçlarımızı geliştirmez,iyileştirmez ama özelliklerin farkında olduğumuzda, özelliklerin bilincinde olduğumuzda ve uygun bir girdi kullandığımızda, bazı sonuçlara daha kolay ulaşabiliriz.Dönüşüm veya normalizasyon gibi çeşitli veri madenciliği tekniklerini denenmiştir ama sonunda, sadece binning kullanılmaya karar verilmiştir ve df_dm adlı yeni bir dataframe oluşturulmuştur.

In [ ]:
df_dm=df.copy()
df_dm.describe()

## Binning - Gruplama

Veri gruplaması, küçük gözlem hatalarının etkilerini azaltmak için kullanılan bir ön işleme tekniğidir. Bu veri setinde bazı sütünlara uygulamaya değerdir. Gruplama yr_built ve yr_renovated özelliğine uygulanmıştır. Evlerin yaşları ve satıldıklarındaki onarım yaşları eklenmiştir. Ayrıca, bu sütunlar aralıklara ayrılmıştır ve bunu aşağıdaki histogramda görebilirsiniz.

In [ ]:
# sadece yıl özelliğini alıyoruz
df_dm['sales_yr']=df_dm['date'].astype(str).str[:4]

# Konut satıldığındaki yaşını yeni bir sütun olarak ekliyoruz
df_dm['age']=df_dm['sales_yr'].astype(int)-df_dm['yr_built']

# Konut satıldığındaki yenileme yaşını yeni bir sütun olarak ekliyoruz.
df_dm['age_rnv']=0
df_dm['age_rnv']=df_dm['sales_yr'][df_dm['yr_renovated']!=0].astype(int)-df_dm['yr_renovated'][df_dm['yr_renovated']!=0]
df_dm['age_rnv'][df_dm['age_rnv'].isnull()]=0

# yaşı bölümlere ayırıyoruz.
bins = [-2,0,5,10,25,50,75,100,100000]
labels = ['<1','1-5','6-10','11-25','26-50','51-75','76-100','>100']
df_dm['age_binned'] = pd.cut(df_dm['age'], bins=bins, labels=labels)

# age_rnv özelliğini bölümlere ayırıyoruz.
bins = [-2,0,5,10,25,50,75,100000]
labels = ['<1','1-5','6-10','11-25','26-50','51-75','>75']
df_dm['age_rnv_binned'] = pd.cut(df_dm['age_rnv'], bins=bins, labels=labels)

# bölümlere ayrılan sütunların histogramı
f, axes = plt.subplots(1, 2,figsize=(15,5))
p1=sns.countplot(df_dm['age_binned'],ax=axes[0])
for p in p1.patches:
    height = p.get_height()
    p1.text(p.get_x()+p.get_width()/2,height + 50,height,ha="center")   
p2=sns.countplot(df_dm['age_rnv_binned'],ax=axes[1])
sns.despine(left=True, bottom=True)
for p in p2.patches:
    height = p.get_height()
    p2.text(p.get_x()+p.get_width()/2,height + 200,height,ha="center")
    
axes[0].set(xlabel='Age')
axes[0].yaxis.tick_left()
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='Renovation Age');

# modelde kullanabilmek için faktör değerlerini dönüştürmek
df_dm = pd.get_dummies(df_dm, columns=['age_binned','age_rnv_binned'])

# Çoklu Regresyon (Multiple Regression)

İlk başta basit doğrusal regresyon kullandık ancak zayıf bir uyum(fit) elde ettik. Bu modeli geliştirmek için daha fazla özellik ekleyelim. Doğrusal Regresyonda birden fazla özellik olduğunda buna çoklu regresyon denir. O zaman şimdi daha karmaşık modeller oluşturalım:

## Çoklu Regresyon - 1

İlk önce özellikler önceki bölümlere bakılarak belirlenmiştir ve ilk çoklu doğrusal regresyonda kullanılmıştır.

In [ ]:
# veri seti ,eğitim setine ve test setine bölünüyor
train_data_dm,test_data_dm = train_test_split(df_dm,train_size = 0.8,random_state=3)


features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','zipcode'] #özelliklerin seçimi
complex_model_1 = linear_model.LinearRegression() 
complex_model_1.fit(train_data_dm[features],train_data_dm['price']) #çoklu regresyon oluşturma


pred = complex_model_1.predict(test_data_dm[features]) # oluşturulan model kullanılarak tahminlerin yapılması

# hataların hesaplanması
rmsecm = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred)),'.3f'))
rtrcm = float(format(complex_model_1.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm = float(format(adjustedR2(complex_model_1.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm = float(format(complex_model_1.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm = float(format(adjustedR2(complex_model_1.score(test_data_dm[features],test_data['price']),test_data_dm.shape[0],len(features)),'.3f'))

# cross validation kullanılarak modelin değerlendirilmesi
cv = float(format(cross_val_score(complex_model_1,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))

r = degerlendirme_tablosu.shape[0]
degerlendirme_tablosu.loc[r] = ['Çoklu Regresyon-1','seçili özellikler',rmsecm,rtrcm,artrcm,rtecm,artecm,cv]
degerlendirme_tablosu.sort_values(by = '5-Fold Cross Validation', ascending=False)

# Çoklu Regresyon - 2

Önceki çoklu regresyona ek olarak özellikler listesine daha fazla özellik eklenmiştir. Değerlendirme metriklerine bakıldığında,model önemli ölçüde geliştiğimtir,iyileşmiştir.

In [ ]:
# özellik seçimi
features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view',
             'grade','age_binned_<1', 'age_binned_1-5', 'age_binned_6-10','age_binned_11-25', 
             'age_binned_26-50', 'age_binned_51-75','age_binned_76-100', 'age_binned_>100',
             'zipcode']

# model oluşturma
complex_model_2 = linear_model.LinearRegression()
complex_model_2.fit(train_data_dm[features],train_data_dm['price'])



pred = complex_model_2.predict(test_data_dm[features]) #tahminler

# hataların hesaplanması
rmsecm = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred)),'.3f'))
rtrcm = float(format(complex_model_2.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm = float(format(adjustedR2(complex_model_2.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm = float(format(complex_model_2.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm = float(format(adjustedR2(complex_model_2.score(test_data_dm[features],test_data_dm['price']),test_data_dm.shape[0],len(features)),'.3f'))

#çapraz doğrulama kullanılarak modelin değerlendirilmesi
cv = float(format(cross_val_score(complex_model_2,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))

r = degerlendirme_tablosu.shape[0]
degerlendirme_tablosu.loc[r] = ['Çoklu Regresyon-2','seçili özellikler',rmsecm,rtrcm,artrcm,rtecm,artecm,cv]
degerlendirme_tablosu.sort_values(by = '5-Fold Cross Validation', ascending=False)

# Çoklu Regresyon - 3

Farklılıkları kolayca gözlemlemek için, herhangi bir ön işleme yapmadan tüm özelliklere sahip bir model oluşturulmuştur. Değerlendirme metrikleri yine dikkat çekici şekilde iyileşti.

In [ ]:
# özellik seçimi
features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view',
            'condition','grade','sqft_above','sqft_basement','yr_built','yr_renovated',
            'zipcode','lat','long','sqft_living15','sqft_lot15']

# model oluşturma
complex_model_3 = linear_model.LinearRegression()
complex_model_3.fit(train_data[features],train_data['price'])


pred = complex_model_3.predict(test_data[features]) #tahminler

# hataların hesaplanması
rmsecm = float(format(np.sqrt(metrics.mean_squared_error(test_data['price'],pred)),'.3f'))
rtrcm = float(format(complex_model_3.score(train_data[features],train_data['price']),'.3f'))
artrcm = float(format(adjustedR2(complex_model_3.score(train_data[features],train_data['price']),train_data.shape[0],len(features)),'.3f'))
rtecm = float(format(complex_model_3.score(test_data[features],test_data['price']),'.3f'))
artecm = float(format(adjustedR2(complex_model_3.score(test_data[features],test_data['price']),test_data.shape[0],len(features)),'.3f'))

#çapraz doğrulama kullanarak modelin değerlendirilmesi

cv = float(format(cross_val_score(complex_model_3,df[features],df['price'],cv=5).mean(),'.3f'))

r = degerlendirme_tablosu.shape[0]
degerlendirme_tablosu.loc[r] = ['Çoklu Regresyon-3','tüm özellikler, ön işleme yok',rmsecm,rtrcm,artrcm,rtecm,artecm,cv]
degerlendirme_tablosu.sort_values(by = '5-Fold Cross Validation', ascending=False)

# Çoklu Regresyon - 4

Bu kez ön işleme adımından sonra elde edilen veriler kullanılmıştır.

In [ ]:
# özellik seçimi
features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront',
            'view','condition','grade','sqft_above','sqft_basement','age_binned_<1', 
            'age_binned_1-5', 'age_binned_6-10','age_binned_11-25', 'age_binned_26-50',
            'age_binned_51-75','age_binned_76-100', 'age_binned_>100','age_rnv_binned_<1',
            'age_rnv_binned_1-5', 'age_rnv_binned_6-10', 'age_rnv_binned_11-25',
            'age_rnv_binned_26-50', 'age_rnv_binned_51-75', 'age_rnv_binned_>75',
            'zipcode','lat','long','sqft_living15','sqft_lot15']

#model oluşturma
complex_model_4 = linear_model.LinearRegression()
complex_model_4.fit(train_data_dm[features],train_data_dm['price'])


pred = complex_model_4.predict(test_data_dm[features]) #tahminlerin yapılması

#hataların hesaplanması
rmsecm = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred)),'.3f'))
rtrcm = float(format(complex_model_4.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm = float(format(adjustedR2(complex_model_4.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm = float(format(complex_model_4.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm = float(format(adjustedR2(complex_model_4.score(test_data_dm[features],test_data_dm['price']),test_data_dm.shape[0],len(features)),'.3f'))

#çapraz değerlendirme kullanılarak modelin değerlendirilmesi
cv = float(format(cross_val_score(complex_model_4,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))

r = degerlendirme_tablosu.shape[0]
degerlendirme_tablosu.loc[r] = ['Çoklu Regresyon-4','tüm özellikler',rmsecm,rtrcm,artrcm,rtecm,artecm,cv]
degerlendirme_tablosu.sort_values(by = '5-Fold Cross Validation', ascending=False)

# Düzenlileştirme (Regularization)

Düzenlileştirme,aşırı uyma(overfitting) ve eksik öğrenme(underfitting) sorununu çözmek için tasarlanmıştır. Aşırı uymanın(overfitting) anlamı yüksek varyans ve karmaşık modeldir ve bu durumda genellikle eğitim verileri iyi uyulur(fit) ancak test verilerinde kötü sonuçlar elde edilmesine neden olabilir. Eksik öğrenme ise düşük varyans ve basit modeldir. Bu da kötü sonuçlara neden olabilir. Olası çözümler özellikleri manuel olarak ayarlamak veya ekstra iş yükü getiren bazı model seçim algoritmaları kullanmaktır. Bunun aksine,düzenlileştirmeyi uyguladığımızda tüm özellikler korunur ve model $\theta_{j}$'yi ayarlar. Bu,özellikle çok az kullanışlı özelliğimiz olduğunda işe yarar. Ridge Regresyon,Lasso Regresyon ve Elastic Net düzenlileştirmesini kullanacağız.

  


<b>Ridge Regresyon,Lasso Regresyon ve Elastic Net ne zaman kullanılır?</b>

* En azından biraz düzenlileştirme olması neredeyse her zaman tercih edilir bu sebeple düz Doğrusal Regresyondan kaçınmalısınız.
* Ridge iyi bir varsayılandır ancak eğer sadece birkaç özelliğin faydalı,işe yarar olduğunu düşünüyorsanız Lasso veya Elastic Net tercih etmelisiniz çünkü Lasso ve Elastic Net faydasız,işe yaramaz özelliklerin ağırlıklarını sıfır yapma eğilimindedir.
* Genel olarak Lasso'ya karşı Elastic Net tercih edilir çünkü Lasso, özellik sayısı eğitim örneği sayısından daha fazla olduğunda veya birkaç özellik güçlü bir şekilde ilişkili(correlated) olduğunda düzensiz davranabilir.
* Birçok küçük / orta boyutlu etkiye sahip değişken varsa: Ridge kullanılabilir.
* Orta / büyük etkiye sahip sadece birkaç değişken varsa: Lasso kullanılabilir.

## Ridge Regresyon

Ridge Regresyona L2 düzenlileştirmesi de denir ve bir ceza terimi ekleyerek aşağıdaki denklemi elde ederiz:

$$RSS_{RIDGE} = \sum_{i=1}^{m}(h_{\theta}(x_{i})-y_{i})^{2} + \alpha \sum_{j=1}^{n}\theta^{2}_{j}$$

$\alpha$ değerini değiştirerek düzenlileştirme miktarını kontrol edebiliriz.Eğer $\alpha=0$ ise, Ridge Regresyon bir Doğrusal Regresyon olur. Eğer $\theta$ çok büyük ise tüm ağırlıklar sıfıra çok yakın olacaktır.

Aşağıda farklı $\alpha$ değerleri kullanılarak farklı Ridge Regresyon modelli eğitilmiştir ve sonuçlar değerlendirme tablosuna eklenmiştir.

In [ ]:
# özellik seçimi
features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront',
            'view','condition','grade','sqft_above','sqft_basement','age_binned_<1', 
            'age_binned_1-5', 'age_binned_6-10','age_binned_11-25', 'age_binned_26-50',
            'age_binned_51-75','age_binned_76-100', 'age_binned_>100','age_rnv_binned_<1',
            'age_rnv_binned_1-5', 'age_rnv_binned_6-10', 'age_rnv_binned_11-25',
            'age_rnv_binned_26-50', 'age_rnv_binned_51-75', 'age_rnv_binned_>75',
            'zipcode','lat','long','sqft_living15','sqft_lot15']

# alpha =1 ile model oluşturulması
complex_model_R = linear_model.Ridge(alpha=1)
complex_model_R.fit(train_data_dm[features],train_data_dm['price'])

pred1 = complex_model_R.predict(test_data_dm[features]) # tahminlerin alınması

#hataların hesaplanması
rmsecm1 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred1)),'.3f'))
rtrcm1 = float(format(complex_model_R.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm1 = float(format(adjustedR2(complex_model_R.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm1 = float(format(complex_model_R.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm1 = float(format(adjustedR2(complex_model_R.score(test_data_dm[features],test_data_dm['price']),test_data_dm.shape[0],len(features)),'.3f'))

#çapraz değerlendirme kullanarak modelin değerlendirilmesi
cv1 = float(format(cross_val_score(complex_model_R,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))

# alpha =100 ile Ridge Regresyon oluşturulması
complex_model_R = linear_model.Ridge(alpha=100)
complex_model_R.fit(train_data_dm[features],train_data_dm['price'])

pred2 = complex_model_R.predict(test_data_dm[features]) #tahminlerin hesaplanması

#hataların hesaplanması
rmsecm2 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred2)),'.3f'))
rtrcm2 = float(format(complex_model_R.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm2 = float(format(adjustedR2(complex_model_R.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm2 = float(format(complex_model_R.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm2 = float(format(adjustedR2(complex_model_R.score(test_data_dm[features],test_data_dm['price']),test_data_dm.shape[0],len(features)),'.3f'))

#çapraz doğrulama kullanarak modelin değerlendirilmesi
cv2 = float(format(cross_val_score(complex_model_R,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))

#alpha=1000 ile Ridge Regresyon modelinin oluşturulması
complex_model_R = linear_model.Ridge(alpha=1000)
complex_model_R.fit(train_data_dm[features],train_data_dm['price'])

pred3 = complex_model_R.predict(test_data_dm[features]) #tahminlerin hesaplanması

# hataların hesaplanması
rmsecm3 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred3)),'.3f'))
rtrcm3 = float(format(complex_model_R.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm3 = float(format(adjustedR2(complex_model_R.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm3 = float(format(complex_model_R.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm3 = float(format(adjustedR2(complex_model_R.score(test_data_dm[features],test_data_dm['price']),test_data_dm.shape[0],len(features)),'.3f'))

#çapraz doğrulama kullanarak modelin değerlendirilmesi
cv3 = float(format(cross_val_score(complex_model_R,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))


#sonuçların değerlendirme tablosuna eklenmesi
r = degerlendirme_tablosu.shape[0]
degerlendirme_tablosu.loc[r] = ['Ridge Regresyon','alpha=1, tüm özellikler',rmsecm1,rtrcm1,artrcm1,rtecm1,artecm1,cv1]
degerlendirme_tablosu.loc[r+1] = ['Ridge Regresyon','alpha=100, tüm özellikler',rmsecm2,rtrcm2,artrcm2,rtecm2,artecm2,cv2]
degerlendirme_tablosu.loc[r+2] = ['Ridge Regresyon','alpha=1000, tüm özellikler',rmsecm3,rtrcm3,artrcm3,rtecm3,artecm3,cv3]
degerlendirme_tablosu.sort_values(by = '5-Fold Cross Validation', ascending=False)

## Lasso Regresyon

Lasso Regresyona L1 düzenlileştirilmesi de denir ve aşağıdaki gibi tanımlanır:

$$RSS_{LASSO} = \sum_{i=1}^{m}(h_{\theta}(x_{i})-y_{i})^{2} + \alpha \sum_{j=1}^{n}|\theta_{j}|$$

Ridge Regresyon ile Lasso Regresyon arasındaki temel fark ceza terimidir. $\alpha$ değerinin görevi aynıdır.

In [ ]:
# özellik seçimi
features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront',
            'view','condition','grade','sqft_above','sqft_basement','age_binned_<1', 
            'age_binned_1-5', 'age_binned_6-10','age_binned_11-25', 'age_binned_26-50',
            'age_binned_51-75','age_binned_76-100', 'age_binned_>100','age_rnv_binned_<1',
            'age_rnv_binned_1-5', 'age_rnv_binned_6-10', 'age_rnv_binned_11-25',
            'age_rnv_binned_26-50', 'age_rnv_binned_51-75', 'age_rnv_binned_>75',
            'zipcode','lat','long','sqft_living15','sqft_lot15']

# alpha=1 ile Lasso Regresyon modeli oluşturma
complex_model_L = linear_model.Lasso(alpha=1)
complex_model_L.fit(train_data_dm[features],train_data_dm['price'])

pred1 = complex_model_L.predict(test_data_dm[features]) #tahminlerin hesaplanması

#hataların hesaplanması
rmsecm1 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred1)),'.3f'))
rtrcm1 = float(format(complex_model_L.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm1 = float(format(adjustedR2(complex_model_L.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm1 = float(format(complex_model_L.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm1 = float(format(adjustedR2(complex_model_L.score(test_data_dm[features],test_data_dm['price']),test_data_dm.shape[0],len(features)),'.3f'))

#çapraz doğrulama kullanarak modelin değerlendirilmesi
cv1 = float(format(cross_val_score(complex_model_L,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))

#alpha=100 ile Lasso Regresyon modelinin oluşturulması
complex_model_L = linear_model.Lasso(alpha=100)
complex_model_L.fit(train_data_dm[features],train_data_dm['price'])

pred2 = complex_model_L.predict(test_data_dm[features]) #tahminlerin yapılması

#hataların hesaplanması
rmsecm2 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred2)),'.3f'))
rtrcm2 = float(format(complex_model_L.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm2 = float(format(adjustedR2(complex_model_L.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm2 = float(format(complex_model_L.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm2 = float(format(adjustedR2(complex_model_L.score(test_data_dm[features],test_data_dm['price']),test_data_dm.shape[0],len(features)),'.3f'))

#çapraz doğrulama kullanarak modelin değerlendirilmesi
cv2 = float(format(cross_val_score(complex_model_L,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))

# alpha=1000 ile Lasso Regresyon modelinin oluşturulması
complex_model_L = linear_model.Lasso(alpha=1000)
complex_model_L.fit(train_data_dm[features],train_data_dm['price'])

pred3 = complex_model_L.predict(test_data_dm[features]) #tahminlerin yapılması

#hataların hesaplanması
rmsecm3 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred3)),'.3f'))
rtrcm3 = float(format(complex_model_L.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm3 = float(format(adjustedR2(complex_model_L.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm3 = float(format(complex_model_L.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm3 = float(format(adjustedR2(complex_model_L.score(test_data_dm[features],test_data_dm['price']),test_data_dm.shape[0],len(features)),'.3f'))

#çapraz doğrulama kullanarak modelin değerlendirilmesi
cv3 = float(format(cross_val_score(complex_model_L,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))

#sonuçların değerlendirme tablosuna eklenmesi
r = degerlendirme_tablosu.shape[0]
degerlendirme_tablosu.loc[r] = ['Lasso Regresyon','alpha=1, tüm özellikler',rmsecm1,rtrcm1,artrcm1,rtecm1,artecm1,cv1]
degerlendirme_tablosu.loc[r+1] = ['Lasso Regresyon','alpha=100, tüm özellikler',rmsecm2,rtrcm2,artrcm2,rtecm2,artecm2,cv2]
degerlendirme_tablosu.loc[r+2] = ['Lasso Regresyon','alpha=1000, tüm özellikler',rmsecm3,rtrcm3,artrcm3,rtecm3,artecm3,cv3]
degerlendirme_tablosu.sort_values(by = '5-Fold Cross Validation', ascending=False)

# Elastic Net

Elastic Net, Ridge Regresyon ile Lasso Regresyonun ortasındadır. Elastic Net'in maliyet fonksiyonu Ridge Regresyonun maliyet fonksiyonu ile Lasso Regresyonun maliyet fonksiyonunun karışımıdır ve bu karışımı,r hiperparametresini kullanarak kontrol edebilirsiniz. r=0 olduğunda Elastic Net, Ridge Regresyona denktir ve r=1 olduğunda Elastic Net, Lasso Regresyona denktir.

Elastic Net Maliyet Fonksiyonu:

$$J(\theta)=MSE(\theta)+r \alpha \sum_{i=1}^n |\theta_{i}|+\frac{1-r}{2} \alpha \sum_{i=1}^{n} \theta_{i}^{2}$$

In [ ]:
# özellik seçimi
features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront',
            'view','condition','grade','sqft_above','sqft_basement','age_binned_<1', 
            'age_binned_1-5', 'age_binned_6-10','age_binned_11-25', 'age_binned_26-50',
            'age_binned_51-75','age_binned_76-100', 'age_binned_>100','age_rnv_binned_<1',
            'age_rnv_binned_1-5', 'age_rnv_binned_6-10', 'age_rnv_binned_11-25',
            'age_rnv_binned_26-50', 'age_rnv_binned_51-75', 'age_rnv_binned_>75',
            'zipcode','lat','long','sqft_living15','sqft_lot15']

# alpha=1 ve l1_ratio=0.5 ile Elastic Net Regresyon modeli oluşturma
complex_model_E = linear_model.ElasticNet(alpha=1,l1_ratio=0.5)
complex_model_E.fit(train_data_dm[features],train_data_dm['price'])

pred1 = complex_model_E.predict(test_data_dm[features]) #tahminlerin hesaplanması

#hataların hesaplanması
rmsecm1 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred1)),'.3f'))
rtrcm1 = float(format(complex_model_E.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm1 = float(format(adjustedR2(complex_model_E.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm1 = float(format(complex_model_E.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm1 = float(format(adjustedR2(complex_model_E.score(test_data_dm[features],test_data_dm['price']),test_data_dm.shape[0],len(features)),'.3f'))

#çapraz doğrulama kullanarak modelin değerlendirilmesi
cv1 = float(format(cross_val_score(complex_model_E,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))

#alpha=100 ve l1_ratio=0.5 ile Elastic Net Regresyon modelinin oluşturulması
complex_model_E = linear_model.ElasticNet(alpha=100,l1_ratio=0.5)
complex_model_E.fit(train_data_dm[features],train_data_dm['price'])

pred2 = complex_model_E.predict(test_data_dm[features]) #tahminlerin yapılması

#hataların hesaplanması
rmsecm2 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred2)),'.3f'))
rtrcm2 = float(format(complex_model_E.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm2 = float(format(adjustedR2(complex_model_E.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm2 = float(format(complex_model_E.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm2 = float(format(adjustedR2(complex_model_E.score(test_data_dm[features],test_data_dm['price']),test_data_dm.shape[0],len(features)),'.3f'))

#çapraz doğrulama kullanarak modelin değerlendirilmesi
cv2 = float(format(cross_val_score(complex_model_E,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))

# alpha=1000 l1_ratio=0.5 ile Elastic Net Regresyon modelinin oluşturulması
complex_model_E = linear_model.ElasticNet(alpha=1000,l1_ratio=0.5)
complex_model_E.fit(train_data_dm[features],train_data_dm['price'])

pred3 = complex_model_E.predict(test_data_dm[features]) #tahminlerin yapılması

#hataların hesaplanması
rmsecm3 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred3)),'.3f'))
rtrcm3 = float(format(complex_model_E.score(train_data_dm[features],train_data_dm['price']),'.3f'))
artrcm3 = float(format(adjustedR2(complex_model_E.score(train_data_dm[features],train_data_dm['price']),train_data_dm.shape[0],len(features)),'.3f'))
rtecm3 = float(format(complex_model_E.score(test_data_dm[features],test_data_dm['price']),'.3f'))
artecm3 = float(format(adjustedR2(complex_model_E.score(test_data_dm[features],test_data_dm['price']),test_data_dm.shape[0],len(features)),'.3f'))

#çapraz doğrulama kullanarak modelin değerlendirilmesi
cv3 = float(format(cross_val_score(complex_model_E,df_dm[features],df_dm['price'],cv=5).mean(),'.3f'))

#sonuçların değerlendirme tablosuna eklenmesi
r = degerlendirme_tablosu.shape[0]
degerlendirme_tablosu.loc[r] = ['Elastic Net Regresyon','alpha=1 ve l1_ratio=0.5, tüm özellikler',rmsecm1,rtrcm1,artrcm1,rtecm1,artecm1,cv1]
degerlendirme_tablosu.loc[r+1] = ['Elastic Net Regresyon','alpha=100 ve l1_ratio=0.5, tüm özellikler',rmsecm2,rtrcm2,artrcm2,rtecm2,artecm2,cv2]
degerlendirme_tablosu.loc[r+2] = ['Elastic Net Regresyon','alpha=1000 ve l1_ratio=0.5, tüm özellikler',rmsecm3,rtrcm3,artrcm3,rtecm3,artecm3,cv3]
degerlendirme_tablosu.sort_values(by = '5-Fold Cross Validation', ascending=False)

# Polinom Regresyon

Doğrusal modeller için ana fikir verilerimize düz bir çizgi uydurmaktır(fit). Ancak veriler ikinci dereceden dağılıma sahipse, ikinci dereceden bir fonksiyon seçmek ve bir polinom dönüşüşümü uygulamak bize daha iyi sonuçlar verebilir.Bu kez hipotez fonksiyonu şu şekilde tanımlanır:

$$h_{\theta}(X)=\theta_{0}+\theta_{1}x+\theta_{2}x^{2}+...+\theta_{n}x^{n}$$

Polinom Regresyonu için çok çeşitlilik olduğu için,sonuçları yeni bir tablo ile göstermek tercih edilmiştir ve aşağıdaki tabloda polinom dönüşümünün modelin aşırı uymasını(overfitting) iyileştirdiği görülebilir. Öte yanda, polinom dönüşümü kullanırken ve dereceye karar verirken dikkatli olmalıyız çünkü aşırı uymaya(overfitting) neden olabilir. Ayrıca,aşağıdaki tabloda bazı modeller için aşırı uyma(overfitting) vardır. 5-fold cross validation metrikleri,eğitim seti için çok yüksek R kare değerlerine sahip olmalarına rağmen bu modeller için(aşırı uymuş(overfitting) modeller için) negatif veya düşüktür.

In [ ]:
evaluation_poly = pd.DataFrame({'Model': [],
                                'Detaylar':[],
                                'Root Mean Squared Error (RMSE)':[],
                                'R-squared (eğitim)':[],
                                'Adjusted R-squared (eğitim)':[],
                                'R-squared (test)':[],
                                'Adjusted R-squared (test)':[],
                                '5-Fold Cross Validation':[]})

features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view',
             'grade','yr_built','zipcode']
polyfeat = PolynomialFeatures(degree=2)
X_allpoly = polyfeat.fit_transform(df[features])
X_trainpoly = polyfeat.fit_transform(train_data[features])
X_testpoly = polyfeat.fit_transform(test_data[features])
poly = linear_model.LinearRegression().fit(X_trainpoly, train_data['price'])

pred1 = poly.predict(X_testpoly)
rmsepoly1 = float(format(np.sqrt(metrics.mean_squared_error(test_data['price'],pred1)),'.3f'))
rtrpoly1 = float(format(poly.score(X_trainpoly,train_data['price']),'.3f'))
rtepoly1 = float(format(poly.score(X_testpoly,test_data['price']),'.3f'))
cv1 = float(format(cross_val_score(linear_model.LinearRegression(),X_allpoly,df['price'],cv=5).mean(),'.3f'))

polyfeat = PolynomialFeatures(degree=3)
X_allpoly = polyfeat.fit_transform(df[features])
X_trainpoly = polyfeat.fit_transform(train_data[features])
X_testpoly = polyfeat.fit_transform(test_data[features])
poly = linear_model.LinearRegression().fit(X_trainpoly, train_data['price'])

pred2 = poly.predict(X_testpoly)
rmsepoly2 = float(format(np.sqrt(metrics.mean_squared_error(test_data['price'],pred2)),'.3f'))
rtrpoly2 = float(format(poly.score(X_trainpoly,train_data['price']),'.3f'))
rtepoly2 = float(format(poly.score(X_testpoly,test_data['price']),'.3f'))
cv2 = float(format(cross_val_score(linear_model.LinearRegression(),X_allpoly,df['price'],cv=5).mean(),'.3f'))

features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view',
            'condition','grade','sqft_above','sqft_basement','yr_built','yr_renovated',
            'zipcode','lat','long','sqft_living15','sqft_lot15']
polyfeat = PolynomialFeatures(degree=2)
X_allpoly = polyfeat.fit_transform(df[features])
X_trainpoly = polyfeat.fit_transform(train_data[features])
X_testpoly = polyfeat.fit_transform(test_data[features])
poly = linear_model.LinearRegression().fit(X_trainpoly, train_data['price'])

pred3 = poly.predict(X_testpoly)
rmsepoly3 = float(format(np.sqrt(metrics.mean_squared_error(test_data['price'],pred3)),'.3f'))
rtrpoly3 = float(format(poly.score(X_trainpoly,train_data['price']),'.3f'))
rtepoly3 = float(format(poly.score(X_testpoly,test_data['price']),'.3f'))
cv3 = float(format(cross_val_score(linear_model.LinearRegression(),X_allpoly,df['price'],cv=5).mean(),'.3f'))

polyfeat = PolynomialFeatures(degree=3)
X_allpoly = polyfeat.fit_transform(df[features])
X_trainpoly = polyfeat.fit_transform(train_data[features])
X_testpoly = polyfeat.fit_transform(test_data[features])
poly = linear_model.LinearRegression().fit(X_trainpoly, train_data['price'])

pred4 = poly.predict(X_testpoly)
rmsepoly4 = float(format(np.sqrt(metrics.mean_squared_error(test_data['price'],pred4)),'.3f'))
rtrpoly4 = float(format(poly.score(X_trainpoly,train_data['price']),'.3f'))
rtepoly4 = float(format(poly.score(X_testpoly,test_data['price']),'.3f'))
cv4 = float(format(cross_val_score(linear_model.LinearRegression(),X_allpoly,df['price'],cv=5).mean(),'.3f'))


features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront',
            'view','condition','grade','sqft_above','sqft_basement','age_binned_<1', 
            'age_binned_1-5', 'age_binned_6-10','age_binned_11-25', 'age_binned_26-50',
            'age_binned_51-75','age_binned_76-100', 'age_binned_>100','age_rnv_binned_<1',
            'age_rnv_binned_1-5', 'age_rnv_binned_6-10', 'age_rnv_binned_11-25',
            'age_rnv_binned_26-50', 'age_rnv_binned_51-75', 'age_rnv_binned_>75',
            'zipcode','lat','long','sqft_living15','sqft_lot15']
polyfeat = PolynomialFeatures(degree=2)
X_allpoly = polyfeat.fit_transform(df_dm[features])
X_trainpoly = polyfeat.fit_transform(train_data_dm[features])
X_testpoly = polyfeat.fit_transform(test_data_dm[features])
poly = linear_model.LinearRegression().fit(X_trainpoly, train_data['price'])

pred5 = poly.predict(X_testpoly)
rmsepoly5 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred5)),'.3f'))
rtrpoly5 = float(format(poly.score(X_trainpoly,train_data_dm['price']),'.3f'))
rtepoly5 = float(format(poly.score(X_testpoly,test_data_dm['price']),'.3f'))
cv5 = float(format(cross_val_score(linear_model.LinearRegression(),X_allpoly,df_dm['price'],cv=5).mean(),'.3f'))

polyfeat = PolynomialFeatures(degree=2)
X_allpoly = polyfeat.fit_transform(df_dm[features])
X_trainpoly = polyfeat.fit_transform(train_data_dm[features])
X_testpoly = polyfeat.fit_transform(test_data_dm[features])
poly = linear_model.Ridge(alpha=1).fit(X_trainpoly, train_data['price'])

pred6 = poly.predict(X_testpoly)
rmsepoly6 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred6)),'.3f'))
rtrpoly6 = float(format(poly.score(X_trainpoly,train_data_dm['price']),'.3f'))
rtepoly6 = float(format(poly.score(X_testpoly,test_data_dm['price']),'.3f'))
cv6 = float(format(cross_val_score(linear_model.Ridge(alpha=1),X_allpoly,df_dm['price'],cv=5).mean(),'.3f'))

polyfeat = PolynomialFeatures(degree=2)
X_allpoly = polyfeat.fit_transform(df_dm[features])
X_trainpoly = polyfeat.fit_transform(train_data_dm[features])
X_testpoly = polyfeat.fit_transform(test_data_dm[features])
poly = linear_model.Ridge(alpha=50000).fit(X_trainpoly, train_data['price'])

pred7 = poly.predict(X_testpoly)
rmsepoly7 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred7)),'.3f'))
rtrpoly7 = float(format(poly.score(X_trainpoly,train_data_dm['price']),'.3f'))
rtepoly7 = float(format(poly.score(X_testpoly,test_data_dm['price']),'.3f'))
cv7 = float(format(cross_val_score(linear_model.Ridge(alpha=50000),X_allpoly,df_dm['price'],cv=5).mean(),'.3f'))

polyfeat = PolynomialFeatures(degree=2)
X_allpoly = polyfeat.fit_transform(df_dm[features])
X_trainpoly = polyfeat.fit_transform(train_data_dm[features])
X_testpoly = polyfeat.fit_transform(test_data_dm[features])
poly = linear_model.Lasso(alpha=1).fit(X_trainpoly, train_data['price'])

pred8 = poly.predict(X_testpoly)
rmsepoly8 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred8)),'.3f'))
rtrpoly8 = float(format(poly.score(X_trainpoly,train_data_dm['price']),'.3f'))
rtepoly8 = float(format(poly.score(X_testpoly,test_data_dm['price']),'.3f'))
cv8 = float(format(cross_val_score(linear_model.Lasso(alpha=1),X_allpoly,df_dm['price'],cv=5).mean(),'.3f'))

polyfeat = PolynomialFeatures(degree=2)
X_allpoly = polyfeat.fit_transform(df_dm[features])
X_trainpoly = polyfeat.fit_transform(train_data_dm[features])
X_testpoly = polyfeat.fit_transform(test_data_dm[features])
poly = linear_model.Lasso(alpha=50000).fit(X_trainpoly, train_data['price'])

pred9 = poly.predict(X_testpoly)
rmsepoly9 = float(format(np.sqrt(metrics.mean_squared_error(test_data_dm['price'],pred9)),'.3f'))
rtrpoly9 = float(format(poly.score(X_trainpoly,train_data_dm['price']),'.3f'))
rtepoly9 = float(format(poly.score(X_testpoly,test_data_dm['price']),'.3f'))
cv9 = float(format(cross_val_score(linear_model.Lasso(alpha=50000),X_allpoly,df_dm['price'],cv=5).mean(),'.3f'))

r = evaluation_poly.shape[0]
evaluation_poly.loc[r] = ['Polynomial Regression','degree=2, selected features, no preprocessing',rmsepoly1,rtrpoly1,'-',rtepoly1,'-',cv1]
evaluation_poly.loc[r+1] = ['Polynomial Regression','degree=3, selected features, no preprocessing',rmsepoly2,rtrpoly2,'-',rtepoly2,'-',cv2]
evaluation_poly.loc[r+2] = ['Polynomial Regression','degree=2, all features, no preprocessing',rmsepoly3,rtrpoly3,'-',rtepoly3,'-',cv3]
evaluation_poly.loc[r+3] = ['Polynomial Regression','degree=3, all features, no preprocessing',rmsepoly4,rtrpoly4,'-',rtepoly4,'-',cv4]
evaluation_poly.loc[r+4] = ['Polynomial Regression','degree=2, all features',rmsepoly5,rtrpoly5,'-',rtepoly5,'-',cv5]
evaluation_poly.loc[r+5] = ['Polynomial Ridge Regression','alpha=1, degree=2, all features',rmsepoly6,rtrpoly6,'-',rtepoly6,'-',cv6]
evaluation_poly.loc[r+6] = ['Polynomial Ridge Regression','alpha=50000, degree=2, all features',rmsepoly7,rtrpoly7,'-',rtepoly7,'-',cv7]
evaluation_poly.loc[r+7] = ['Polynomial Lasso Regression','alpha=1, degree=2, all features',rmsepoly8,rtrpoly8,'-',rtepoly8,'-',cv8]
evaluation_poly.loc[r+8] = ['Polynomial Lasso Regression','alpha=50000, degree=2, all features',rmsepoly9,rtrpoly9,'-',rtepoly9,'-',cv9]
evaluation_poly_temp = evaluation_poly[['Model','Details','Root Mean Squared Error (RMSE)','R-squared (training)','R-squared (test)','5-Fold Cross Validation']]
evaluation_poly_temp.sort_values(by = '5-Fold Cross Validation', ascending=False)

# Değerlendirme Tablosu

In [ ]:
degerlendirme_tablosu_temp=degerlendirme_tablosu.append(evaluation_poly)
degerlendirme_tablosu_temp1=degerlendirme_tablosu_temp.sort_values(by = '5-Fold Cross Validation', ascending=False)
degerlendirme_tablosu_temp2=degerlendirme_tablosu_temp1.reset_index()
degerlendirme_tablosu_f=degerlendirme_tablosu_temp2.iloc[:,1:]
degerlendirme_tablosu_f

Çalışırken Türkeleştirmeye çalıştığım biraz ekleme yaptığım kernel.
<br>
Kaynak = https://www.kaggle.com/burhanykiyakoglu/predicting-house-prices